### Metric 1: Average commutator norm

In [ ]:
import json
import numpy as np
from ansatz import Ansatz
import pennylane as qml
from twirler.symmetry_groups import create_symmetric_group, create_induced_subgroup_from_permutations
from twirler.generators import get_ansatz_generators
from twirler.induced_representation import derive_unitaries_angle_embedding_analytic

n_qubits = 5

S = create_symmetric_group(n_qubits)

subgroups = None
with open(f"groups\\modified_subgroup_generators_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

(0, 1, 2, 3, 4)


In [ ]:
subgroup_unitaries = {}
for k, groups in subgroups.items():
    subgroup_unitaries[k] = []
    for generators in groups:
        new_generators = []
        for g in generators:
            new_generators.append(tuple(g))
        K = create_induced_subgroup_from_permutations(S, new_generators)
        unitaries = derive_unitaries_angle_embedding_analytic(K)
        subgroup_unitaries[k].append(unitaries)

In [ ]:
results = {}
depth = 1
for ansatz_id in range(1, 20):
    results[ansatz_id] = {}
    super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
    ansatz = super_ansatz.get_ansatz()
    generators = get_ansatz_generators(ansatz)

    commuted_generators = {}

    for k in subgroup_unitaries:
        total_norm = 0.0
        for unitary in subgroup_unitaries[k]:
            for elem in unitary:
                U_s = unitary[elem]
            
                for gen_idx, (gen_observable, wires, gate_name, theta) in enumerate(generators):
                
                    op = qml.Hermitian(gen_observable, wires=wires)
                    G_full = qml.matrix(op, wire_order=range(n_qubits))

                    commuter = U_s @ G_full - G_full @ U_s
                    norm = np.linalg.norm(commuter, ord='fro')
                    total_norm += norm

        avg_norm = total_norm / (len(generators) * len(unitary) * len(subgroup_unitaries[k]))
        results[ansatz_id][k] = avg_norm

In [4]:
for ansatz_id in results:
    print(f"Ansatz {ansatz_id}:")
    for s in results[ansatz_id]:
        print(f"  Subgroup size {s}: Average commutator norm = {results[ansatz_id][s]}")

Ansatz 1:
  Subgroup size 1: Average commutator norm = 0.0
  Subgroup size 2: Average commutator norm = 1.28
  Subgroup size 3: Average commutator norm = 1.6
  Subgroup size 4: Average commutator norm = 2.0
  Subgroup size 5: Average commutator norm = 3.2
  Subgroup size 6: Average commutator norm = 2.1333333333333333
  Subgroup size 8: Average commutator norm = 2.4
  Subgroup size 10: Average commutator norm = 3.2
  Subgroup size 12: Average commutator norm = 2.4
  Subgroup size 20: Average commutator norm = 3.2
  Subgroup size 24: Average commutator norm = 2.4
  Subgroup size 60: Average commutator norm = 3.2
  Subgroup size 120: Average commutator norm = 3.2
Ansatz 2:
  Subgroup size 1: Average commutator norm = 0.0
  Subgroup size 2: Average commutator norm = 1.698081618838079
  Subgroup size 3: Average commutator norm = 2.1852520988148747
  Subgroup size 4: Average commutator norm = 2.6421305426919006
  Subgroup size 5: Average commutator norm = 3.8693035954915493
  Subgroup size 